In [13]:
from library import table, append_table, historize, get_clubs, get_competitions, load_keys, headers, Historize
import sqlite3
from datetime import datetime, timedelta
import pandas as pd
import requests
from bs4 import BeautifulSoup
import hashlib

### Leagues

In [14]:
leagues_history = table("league")

league_data = get_competitions()
leagues =  pd.DataFrame(league_data, columns =  ['name', 'link'])
leagues = pd.merge(leagues.assign(joined =  1), leagues_history, on =  ['name', 'link'], how = 'outer')
leagues = leagues[leagues.joined != 1]

if len(leagues > 0):
    print(f"Committing {len(leagues)} records")
    leagues = leagues.drop("joined", axis = 1)
    append_table(leagues, league)
else:
    print("League table up-to-date")

League table up-to-date


### Clubs

In [15]:
leagues = table("league")
league_urls = leagues.link.unique()

club_df = pd.DataFrame()

# Scrape clubs
for url in league_urls:
    club_df = pd.concat([club_df, get_clubs(url)])

# deduplicate, clean columns
club_df = club_df.dropna(subset = 'Club.1')
selection = ['Club.1', 'link']
club_df = club_df[selection].drop_duplicates().rename({"Club.1" : "club"}, axis = 1)

# Add hash keys and effective dates
club_df = historize(club_df)

# Filter only new data
current_clubs = table("club")
updates = club_df[~club_df.hash_key.isin(current_clubs.hash_key.unique())]

#Commit changes
if len(updates) > 0:
    print(f"Adding {len(updates)} new club records")
    append_table(updates ,'club')
else:
    print("Club table up-to-date")

Request failed with status code 500 (https://www.transfermarkt.com/persian-gulf-pro-league/startseite/wettbewerb/IRN1)
Club table up-to-date


### Club Links

In [94]:
clubs = load_keys('club', ['club', 'link']) 

league_urls = clubs.link.unique()

club_links = []

for idx, league_url in enumerate(league_urls):
    if idx // 7 == idx / 7:
        print(f"{round(100*idx/len(league_urls))}% links retrieved\n")
        
    response = requests.get(league_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the teams
    temp = soup.select('table.items tbody tr')
    for row in soup.select('table.items tbody tr'):
        link_tag = row.select_one('td.hauptlink a')
        if link_tag:
            club_name = link_tag.text.strip()
            relative_link = link_tag['href']
            #filter out top scorer links
            if r"profil/spieler" not in relative_link:
                full_link = "https://www.transfermarkt.com" + relative_link
                club_links.append((league_url, club_name, full_link))
            

0% complete
https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1
https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1
https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1
21% complete
https://www.transfermarkt.com/liga-portugal/startseite/wettbewerb/PO1
https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1
https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1
https://www.transfermarkt.com/super-liga-srbije/startseite/wettbewerb/SER1
https://www.transfermarkt.com/super-league-1/startseite/wettbewerb/GR1
42% complete
https://www.transfermarkt.com/allsvenskan/startseite/wettbewerb/SE1
https://www.transfermarkt.com/supersport-hnl/startseite/wettbewerb/KR1
https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1
https://www.transfermarkt.com/campeonato-brasileiro-serie-a/

In [95]:
# deduplicate / filter links
club_links_df = pd.DataFrame(club_links, columns = ['league_link', 'club', 'club_link'])
club_links_df = club_links_df[club_links_df.club_link.str.contains("start")] 

In [96]:
historizer = Historize(
    club_links_df,
    primary_key = ['league_link', 'club']
)
historizer.run()
updates = historizer.df
updates = updates.rename({"ID": "club_link_id"}, axis = 1)

Are you sure you want to overwrite this table? This will overwrite the existing data permanently (y/n)
y


In [93]:
def commitChanges(df, table_name, primary_key, db_path = "transfermarkt.db"):
    
    yesterday = str((datetime.today() - timedelta(days=1)).date())

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT DISTINCT hash_key FROM {table_name}")
    existing_keys = [row[0] for row in cursor.fetchall()]
    
    update_df = df[~df.hash_key.isin(existing_keys)]
    update_keys = update_df[primary_key].unique()
    conn.close()
    
    print(f"{len(update_keys)} records being updated - effective_end_date == {yesterday}\n")
    for key in update_keys:
        
        effective_date_update_query = f"""UPDATE {table_name}
        SET effective_end_date = '{yesterday}'
        WHERE {primary_key} = '{key}'
        """
        conn = sqlite3.connect('transfermarkt.db')
        cursor = conn.cursor()
        cursor.execute(effective_date_update_query)
        conn.commit()

        
        conn.close()
    
    #append rows
    print("Appending rows\n")
    conn = sqlite3.connect(db_path)
    update_df.to_sql(table_name, conn, if_exists="append", index=False)
    conn.close()
    
    print("Done!")

commitChanges(
    df = updates, 
    table_name = 'club_link', 
    primary_key='club_link_id'
)

0 records being updated - effective_end_date == 2025-04-29

Appending rows

Done!


### Harmonization code

In [ ]:
# def createDetailedURL(url):
#     return url.replace('startseite', 'kader') + r"/plus/1"

# club_links_df["club_link_detailed"] = club_links_df['club_link'].apply(lambda x: createDetailedURL(x))